# Toxicity Classification: Word2Vec+TFIDF
## Table of Contents
* [1. Overview](#1.)
* [2. Configuration](#2.)
* [3. Setup](#3.)
* [4. Import datasets](#4.)
* [5. EDA & Preprocessing](#5.)
    * [5.1 Text Preprocessing Function](#5.1)
    * [5.2 TF-IDF Vectorization](#5.2)
    * [5.3 Word2Vec Vectorization](#5.3)
    * [5.4 Train Validation Split](#5.4)
    * [5.5 Create TensorFlow Dataset](#5.5)
* [6. Model Development](#6.)
    * [6.1 FNet Encoder](#6.1)
    * [6.2 Positional Embedding](#6.2)
    * [6.3 Word2Vec FNet Model](#6.3)
    * [6.4 TFIDF DNN Model](#6.4)
    * [6.5 The Whole Model](#6.5)
    * [6.6 Model Training](#6.6)
    * [6.7 Evaluation](#6.7)
* [7. Submission](#7.)
* [8. References](#8.)

<font color="red" size="3">If you found it useful and would like to back me up, just upvote.</font>

<a id="1."></a>
## 1. Overview
In this notebook, I am going to build a Jigsaw Toxicity Classification Model with FNet using Word2Vec Vectorization and DNN using TFIDF Vectorization. 

I will keep track of three Model: Model with Best Accuracy, Model with Best AUC, and Latest Model. So I can use them for inference and try different ensemble method to get a better score.

Many people like to create a training notebook and inference notebook to separate training and inference process. But to me I think that maintaining two notebooks is very inconvenient, instead I create an output dataset for this notebook. And I add two modes to this notebook, training and inference. During training mode, this notebook is responsible for training, it also infernece using output of the training. Everytime we finish training, we can choose the output to update the version of this output dataset. During inference mode, this notebook will skip training and inference using output dataset directly.

<a id="2."></a>
## 2. Configuration

In [ ]:
class Config:
    vocab_size = 15000 # Vocabulary Size
    tfidf_vocab_size = 20000
    sequence_length = 100 # Length of sequence
    batch_size = 1024
    validation_split = 0.15
    embed_dim = 256
    latent_dim = 256
    epochs = 10 # Number of Epochs to train
    best_auc_model_path = "model_best_auc.tf"
    best_acc_model_path = "model_best_acc.tf"
    lastest_model_path = "model_latest.tf"
    output_dataset_path = "../input/toxicity-classification-word2vectfidf-output"
    labels = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
    modes = ["training", "inference"]
    mode = modes[0]
config = Config()

<a id="3."></a>
## 3. Setup

In [ ]:
import pandas as pd
import tensorflow as tf
import pathlib
import random
import string
import re
import sys
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
import os
import sklearn
import seaborn as sns
from sklearn.model_selection import train_test_split
from nltk.tokenize import TweetTokenizer 
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from scipy.stats import rankdata
import json

<a id="4."></a>
## 4. Import datasets

In [ ]:
!unzip ../input/jigsaw-toxic-comment-classification-challenge/train.csv.zip
!unzip ../input/jigsaw-toxic-comment-classification-challenge/test.csv.zip
!unzip ../input/jigsaw-toxic-comment-classification-challenge/test_labels.csv.zip
!unzip ../input/jigsaw-toxic-comment-classification-challenge/sample_submission.csv.zip

In [ ]:
train = pd.read_csv("/kaggle/working/train.csv")
train.head()

<a id="5."></a>
## 5. EDA & Preprocessing

<a id="5.1"></a>
### 5.1 Text Preprocessing Function 

In [ ]:
def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    stripped_html = tf.strings.regex_replace(lowercase, "<br />", " ")
    text = tf.strings.regex_replace(
        stripped_html, f"[{re.escape(string.punctuation)}]", ""
    )
    text = tf.strings.regex_replace(text, f"[0-9]+", " ")
    text = tf.strings.regex_replace(text, f"[ ]+", " ")
    text = tf.strings.strip(text)
    return text

<a id="5.2"></a>
### 5.2 TF-IDF Vectorization

In [ ]:
tfidf_vectorizer = layers.TextVectorization(
    standardize=custom_standardization, 
    max_tokens=config.tfidf_vocab_size, 
    output_mode="tf-idf", 
    ngrams=2
)
with tf.device("CPU"):
    # A bug that prevents this from running on GPU for now.
    tfidf_vectorizer.adapt(list(train["comment_text"]))

<a id="5.3"></a>
### 5.3 Word2Vec Vectorization

In [ ]:
word2vec_vectorizer = layers.TextVectorization(
    standardize=custom_standardization, 
    max_tokens=config.vocab_size, 
    output_sequence_length=config.sequence_length
)
with tf.device("CPU"):
    word2vec_vectorizer.adapt(train["comment_text"])

<a id="5.4"></a>
### 5.4 Train Validation Split

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(train["comment_text"], train[config.labels], test_size=config.validation_split)

In [ ]:
X_train.shape, y_train.shape, X_val.shape, y_val.shape

<a id="5.5"></a>
### 5.5 Create TensorFlow Dataset

In [ ]:
def make_dataset(X, y, batch_size, mode):
    dataset = tf.data.Dataset.from_tensor_slices((X, y))
    if mode == "train":
       dataset = dataset.shuffle(256) 
    dataset = dataset.batch(batch_size)
    dataset = dataset.cache().prefetch(16).repeat(1)
    return dataset

In [ ]:
train_ds = make_dataset(X_train, y_train, batch_size=config.batch_size, mode="train")
valid_ds = make_dataset(X_val, y_val, batch_size=config.batch_size, mode="valid")

Let's see what this data look like.

In [ ]:
for batch in train_ds.take(1):
    print(batch)

<a id="6."></a>
## 6. Model Development

<a id="6.1"></a>
### 6.1 FNet Encoder

In [ ]:
class FNetEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, dropout_rate=0.1, **kwargs):
        super(FNetEncoder, self).__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.dense_proj = keras.Sequential(
            [
                layers.Dense(dense_dim, activation="relu"),
                layers.Dense(embed_dim),
            ]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()

    def call(self, inputs):
        # Casting the inputs to complex64
        inp_complex = tf.cast(inputs, tf.complex64)
        # Projecting the inputs to the frequency domain using FFT2D and
        # extracting the real part of the output
        fft = tf.math.real(tf.signal.fft2d(inp_complex))
        proj_input = self.layernorm_1(inputs + fft)
        proj_output = self.dense_proj(proj_input)
       
        layer_norm = self.layernorm_2(proj_input + proj_output)
        return layer_norm

<a id="6.2"></a>
### 6.2 Positional Embedding

In [ ]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, vocab_size, embed_dim, **kwargs):
        super(PositionalEmbedding, self).__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=vocab_size, output_dim=embed_dim
        )
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=embed_dim
        )
        self.sequence_length = sequence_length
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)


<a id="6.3"></a>
### 6.3 Word2Vec FNet Model

In [ ]:
def get_word2vec_model(config, inputs):
    x = word2vec_vectorizer(inputs)
    x = PositionalEmbedding(config.sequence_length, config.vocab_size, config.embed_dim)(x)
    x = FNetEncoder(config.embed_dim, config.latent_dim)(x)
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dropout(0.5)(x)
    for i in range(3):
        x = layers.Dense(100, activation="relu")(x)
        x = layers.Dropout(0.3)(x)
    return x

<a id="6.4"></a>
### 6.4 TFIDF DNN Model

In [ ]:
def get_tfidf_model(config, inputs):
    x = tfidf_vectorizer(inputs)
    x = layers.Dense(256, activation="relu", kernel_regularizer="l2")(x)
    x = layers.Dense(100, activation="relu", kernel_regularizer="l2")(x)
    return x

<a id="6.5"></a>
### 6.5 The Whole Model

In [ ]:
def get_model(config):
    inputs = keras.Input(shape=(None, ), dtype="string", name="inputs")
    word2vec_x = get_word2vec_model(config, inputs)
    tfidf_x = get_tfidf_model(config, inputs)
    x = layers.Concatenate()([word2vec_x, tfidf_x])
    output = layers.Dense(6, activation="sigmoid")(x)
    model = keras.Model(inputs, output, name="model")
    return model

In [ ]:
model = get_model(config)
model.summary()

Let's visualize the Model.

In [ ]:
keras.utils.plot_model(model, show_shapes=True)

<a id="6.6"></a>
### 6.6 Model Training

In [ ]:
model.compile(
    "adam", loss="binary_crossentropy", metrics=["categorical_accuracy", keras.metrics.AUC()]
)

In [ ]:
if config.mode == config.modes[0]:
    acc_checkpoint = keras.callbacks.ModelCheckpoint(config.best_acc_model_path, monitor="val_categorical_accuracy",save_weights_only=True, save_best_only=True)
    auc_checkpoint = keras.callbacks.ModelCheckpoint(config.best_auc_model_path, monitor="val_auc",save_weights_only=True, save_best_only=True)
    early_stopping = keras.callbacks.EarlyStopping(patience=10)
    reduce_lr = keras.callbacks.ReduceLROnPlateau(patience=5, min_delta=1e-4, min_lr=1e-6)
    model.fit(train_ds, epochs=config.epochs, validation_data=valid_ds, callbacks=[acc_checkpoint, auc_checkpoint, reduce_lr])
    model.save_weights(config.lastest_model_path)

<a id="6.7"></a>
### 6.7 Evaluation

#### Classification Report

In [ ]:
if config.mode == config.modes[0]:
    from sklearn.metrics import classification_report
    y_pred = np.array(model.predict(valid_ds) > 0.5, dtype=int)
    cls_report = classification_report(y_val, y_pred)
    print(cls_report)

<a id="7."></a>
## 7. Submission

In [ ]:
test = pd.read_csv("/kaggle/working/test.csv")
test.head()

In [ ]:
sample_submission = pd.read_csv("/kaggle/working/sample_submission.csv")
sample_submission.head()

In [ ]:
scores = []
test_ds = tf.data.Dataset.from_tensor_slices((test["comment_text"])).batch(config.batch_size).cache().prefetch(1)
for path in [config.best_auc_model_path]:
#for path in [config.best_acc_model_path, config.best_auc_model_path, config.lastest_model_path]:
    if config.mode == config.modes[1]:
        path = config.output_dataset_path + "/" + path
    model.load_weights(path)
    score = model.predict(test_ds)
    scores.append(score)
score = np.mean(scores, axis=0)
print(score.shape)

In [ ]:
sample_submission[config.labels] = score
sample_submission.to_csv("submission.csv", index=False)
sample_submission.head()


<a id="8."></a>
## 8. References
- [FNet: Mixing Tokens with Fourier Transforms](https://arxiv.org/abs/2105.03824v3)
- [Attention Is All You Need](https://arxiv.org/abs/1706.03762v5)
- [Text Generation using FNet](https://keras.io/examples/nlp/text_generation_fnet/)
- [English-Spanish Translation: FNet](https://www.kaggle.com/lonnieqin/english-spanish-translation-fnet)